# Decision Tree Regression

In [27]:
from warnings import  filterwarnings
filterwarnings('ignore')

# Data Ingestion

In [28]:
 import pandas as pd 
 df = pd.read_csv('PowerPlant.csv')


In [29]:
df.head()

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


# columns
1. AT - atmospheric Temprature in degree celcius
2. v - vaccum pressure
3. AP - atmospheric pressure
4. RH - relative humidity %
5. PE - power outpur in mega watts (target)

In [30]:
df.shape

(9568, 5)

In [31]:
df.isna().sum(
    
)

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [32]:
df.duplicated().sum()

np.int64(41)